# Infenrence

In [92]:
# imports

from src.utils import utils
from src.modelling import training as train
import numpy as np
import pandas as pd
%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [93]:
config = utils.get_config('config/data_config.ini')

#select model
model_name = 'xgboost'
model_path = utils.get_full_path(config['modelling_paths']['model_output']).joinpath(model_name+'.pkl')
model = utils.load_value(model_path)
model

Pipeline(steps=[('select_optimal_cols',
                 OptimalColumnSelector(optimal_cols_path=PosixPath('/home/aroge/projects/dividend-cut-predictor/models/artifacts/optimal_features.pkl'))),
                ('cat_to_ordinal_cols', ColumnsOrdinalEncoder()),
                ['model',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytr...
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.026692793437760223, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=50,
                               max_leaves=None, min_child_weight=6, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=121, n_jobs=-1,
                               num_parallel_tree=None, random_state=None, ...)]])

In [94]:
raw_data = utils.get_full_path(config['data_paths']['raw_data_path'])
inf_year = int(config['year_limits']['inf_year'])
raw_data = pd.read_csv(raw_data)
inf_raw_data = raw_data[raw_data['year'] == (inf_year-1)]

inf_raw_data.head()

label_col_name = 'dps_change_next_year'
X, _ = train.split_Xy(inf_raw_data, label_col_name=label_col_name)
X


,year,industry,symbol,calendarYear,currentRatio,currentRatio_percentage_change,quickRatio,quickRatio_percentage_change,cashRatio,cashRatio_percentage_change,...,dividendYield,dividendYield_percentage_change,enterpriseValueMultiple,enterpriseValueMultiple_percentage_change,priceFairValue,priceFairValue_percentage_change,interestRate,interestRate_percentage_change,adjDividend,dps_growth
10,2023,Conglomerates,MMM,2021,1.704815,-9.559027,1.043165,-14.902195,0.505147,-13.359828,...,0.033253,-0.909598,15.729376,-12.979421,6.803451,-12.859989,5.02,198.809524,6.00,0.671141
21,2023,Specialty Industrial Machinery,AOS,2021,1.566500,-14.191051,1.131391,-21.328131,0.396228,-38.723265,...,0.012391,-30.862468,19.078414,21.967224,7.492633,56.391484,5.02,198.809524,1.22,7.017544
32,2023,Medical Devices,ABT,2021,1.849599,7.740228,1.277070,12.122625,0.747730,30.202100,...,0.012897,-2.300018,23.954222,-12.035499,6.934722,17.230747,5.02,198.809524,2.04,8.510638
43,2023,Drug Manufacturers - General,ABBV,2021,0.793544,-5.912486,0.562795,-6.766883,0.276922,-6.061460,...,0.038643,-10.545641,11.593723,-68.566197,15.554128,13.865668,5.02,198.809524,5.92,4.964539
54,2023,Information Technology Services,ACN,2022,1.233251,-1.492554,1.122526,-1.491940,0.450243,-13.410160,...,0.013463,28.618827,15.572515,-27.626364,8.256685,-24.524208,5.02,198.809524,4.65,15.384615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5158,2023,Restaurants,YUM,2021,1.082686,7.371121,0.764664,1.330120,0.343463,-21.191732,...,0.014355,-16.852644,22.999301,-13.927588,-4.925525,18.551700,5.02,198.809524,2.42,6.140351
5169,2023,Communication Equipment,ZBRA,2021,0.938333,36.437942,0.606667,62.274566,0.184444,103.218254,...,0.000000,0.000000,26.389714,1.213595,10.660622,11.282129,5.02,198.809524,0.00,0.000000
5180,2023,Medical Devices,ZBH,2021,1.408633,-29.145734,0.472245,-46.448271,0.109029,-65.244356,...,0.007777,21.330579,16.585651,-23.940489,2.032172,-19.943415,5.02,198.809524,0.96,0.000000
5191,2023,Banks - Regional,ZION,2021,43.956359,369.717036,43.956359,369.717036,13.563591,1483.677171,...,0.025841,-29.033809,0.163481,-98.186131,1.353357,50.048818,5.02,198.809524,1.64,3.797468


In [95]:
y_pred_prob = model.predict_proba(X)[:, 1]
y_pred = model.predict(X)

In [96]:
def get_top_n_idx(a, n=2, order='desc'):

    if order == 'asc':

        idx_top_n = np.argpartition(a, n)[:n]
        idx_top_n = idx_top_n[np.argsort(a[idx_top_n])]
    else:
        idx_top_n = np.argpartition(a, -n)[-n:]
        idx_top_n = idx_top_n[np.argsort(a[idx_top_n])][::-1]

    return idx_top_n

n = 50
top_n_pred_idxs = get_top_n_idx(y_pred_prob, n=n, order='desc')

y_pred_prob_top_n = y_pred_prob[top_n_pred_idxs]
y_pred_top_n = y_pred[top_n_pred_idxs]



print(f'top {n} probabilities of cutting dividends for {inf_year}:')
df = X.iloc[top_n_pred_idxs, [1,2]]
df['pred. prob'] = y_pred_prob_top_n
df['pred. class'] = y_pred_top_n
display(df)

top 50 probabilities of cutting dividends for 2024:


,industry,symbol,pred. prob,pred. class
1352,Discount Stores,COST,0.764008,1
1275,Oil & Gas E&P,COP,0.554862,1
3024,Specialty Chemicals,LYB,0.519763,1
4740,Railroads,UNP,0.497694,0
4773,Rental & Leasing Services,URI,0.474314,0
4267,Utilities - Diversified,SRE,0.451521,0
1770,Oil & Gas E&P,EOG,0.405000,0
1484,Oil & Gas E&P,DVN,0.381967,0
1506,Oil & Gas E&P,FANG,0.372404,0
945,Financial Data & Stock Exchanges,CBOE,0.366487,0
